In [3]:
import pandas as pd
import numpy as py
import re

# Movie Dataset

In [2]:
df = pd.read_csv('movies.csv')
df["gen"] = df["genres"].str.split("|") 
df.head(5)

,movieId,title,genres,gen
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),Comedy,[Comedy]


In [3]:
df["title_new"] = df["title"].str[:-6]
df["year"] = df["title"].str[-5:-1]
df.head(3)

,movieId,title,genres,gen,title_new,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]",Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]",Grumpier Old Men,1995


In [4]:
df_new = df.drop(['title', 'genres'], axis = 1)
#df_new.to_csv('movies_new.csv',index=False)

# Rating Dataset

In [4]:
df_1 = pd.read_csv('ratings.csv')

In [5]:
import datetime

In [6]:
df_1['date'] = pd.to_datetime(df_1['timestamp'],unit='s')

In [7]:
#create year and month feature from date
df_1['year'] = pd.DatetimeIndex(df_1['date']).year
df_1['month'] = pd.DatetimeIndex(df_1['date']).month

df_1 = df_1.drop('timestamp', axis = 1)

In [8]:
#Using aggregate function to get the number of rating, averate rating score and std for each movie at each year
gkk = df_1.groupby(['movieId', 'year']).agg({'rating':['count','mean','std']}).reset_index()
#Rename columns
gkk.columns = ['movie_id', 'year', 'count','mean','std']

In [13]:
#get the percentage of # count in last year over # count in first year
#get the median of the average rating score for each movie
import statistics
diction = {}
perce = 0
percentage = []
med = []
#calculate percentage and mean
for i in range(1,len(gkk.movie_id.unique())):
    tmp = gkk[gkk['movie_id'] == i]
    if len(tmp) < 2:
        continue
    percentage.append(tmp['count'].iloc[-1]/tmp['count'].iloc[0])
    med.append(statistics.median(tmp['mean']))

0.2

In [18]:
# calculating the median/mean and choose it 
print(statistics.median(percentage))
print(statistics.median(med))

0.2
3.270878271498418


In [19]:
#create label
for i in range(1,len(gkk.movie_id.unique())):
    tmp = gkk[gkk['movie_id'] == i]
    if len(tmp) < 2:
        continue
    percen = tmp['count'].iloc[-1]/tmp['count'].iloc[0] 
    median = statistics.median(tmp['mean'])
    if percen > 0.2 and median >= 3.2:
        diction[i] = 4
    elif percen < 0.2 and median >= 3.2:
        diction[i] = 3
    elif percen > 0.2 and median < 3.2:
        diction[i] = 2
    else:
        diction[i] = 1

In [22]:
first_year = gkk.loc[gkk.groupby(['movie_id']).year.idxmin()]
first_year.head(5)

,movie_id,year,count,mean,std
0,1,1996,8526,4.119751,0.888026
23,2,1996,6900,3.574058,0.885957
46,3,1996,3303,3.424160,0.958047
69,4,1996,1364,3.049120,1.117364
92,5,1996,4136,3.345261,0.916458


In [23]:
type_df = pd.DataFrame.from_dict(diction,orient='index')
type_df['movie_id'] = type_df.index
whole = pd.merge(type_df,first_year, left_on='movie_id', right_on='movie_id',how = 'outer')
whole['year'] = whole['year'].fillna(0.0).astype(int)
whole.columns=['category','movie_id','year','count','mean','std']
whole['category'] = whole['category'].fillna(0.0).astype(int)

,category,movie_id,year,count,mean,std
0,3,1,1996,8526,4.119751,0.888026
1,1,2,1996,6900,3.574058,0.885957
2,1,3,1996,3303,3.424160,0.958047
3,1,4,1996,1364,3.049120,1.117364
4,1,5,1996,4136,3.345261,0.916458
5,3,6,1996,6041,3.815097,0.897796
6,3,7,1996,3537,3.566865,0.873575
7,1,8,1996,589,3.370119,0.966544
8,1,9,1996,1490,3.224832,0.893105
9,3,10,1996,14326,3.455535,0.841887


In [28]:
whole = whole[whole['category']!=0]
yr = whole['year']
print(min(yr))
print(max(yr))
#the minimum year is 1995 and the maximum year is 2017

1995
2017


In [30]:
train = whole[whole.year <= 2010]
test = whole[whole.year > 2010]

In [25]:
# import statistics

# thresh = ealiest['count'].std()+statistics.mean(ealiest['count'])
# thresh

1504.6894288526105

In [ ]:
train.to_csv('train_dataset.csv')
test.to_csv('test_dataset.csv')